In [1]:
import numpy as np 
import pandas as pd 
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image as Img
from keras import Input
from keras.layers import Dense, Reshape, LeakyReLU, Conv2D, Conv2DTranspose, Flatten, Dropout
from keras.models import Model
from keras.optimizers import RMSprop

In [ ]:
from PIL import Image as pil
from pkg_resources import parse_version
if parse_version(pil.__version__)>=parse_version('10.0.0'):
    Image.ANTIALIAS=Image.LANCZOS

In [ ]:
from PIL import Image
IMAGES_COUNT = 8000
PIC_DIR = f'E:/HKI_Year4/UI_TLCN/CelebFaces_Data/cele_data/'

ORIG_WIDTH = 178
ORIG_HEIGHT = 208
diff = (ORIG_HEIGHT - ORIG_WIDTH) // 2

WIDTH = 128
HEIGHT = 128

crop_rect = (0, diff, ORIG_WIDTH, ORIG_HEIGHT - diff)

images = []
for pic_file in tqdm(os.listdir(PIC_DIR)[:IMAGES_COUNT]):
    pic = Image.open(PIC_DIR + pic_file).crop(crop_rect)
    pic.thumbnail((WIDTH, HEIGHT), Image.ANTIALIAS)
    images.append(np.uint8(pic))

In [ ]:
#Image shape
images = np.array(images) / 255
print(images.shape)

In [ ]:
#Display first 25 images
plt.figure(1, figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(images[i])
    plt.axis('off')
plt.show()

In [ ]:
LATENT_DIM = 32
CHANNELS = 3
def create_generator():
    gen_input = Input(shape=(LATENT_DIM, ))

    x = Dense(128)(gen_input)
    x = LeakyReLU()(x)

    x = Dense(256)(x)
    x = LeakyReLU()(x)

    x = Dense(512)(x)
    x = LeakyReLU()(x)

    x = Dense(1024)(x)
    x = LeakyReLU()(x)

    x = Dense(HEIGHT * WIDTH * CHANNELS, activation='tanh')(x)
    x = Reshape((HEIGHT, WIDTH, CHANNELS))(x)

    generator = Model(gen_input, x)
    return generator

In [ ]:
def create_discriminator():
    disc_input = Input(shape=(HEIGHT, WIDTH, CHANNELS))

    x = Conv2D(256, 3)(disc_input)
    x = LeakyReLU()(x)

    x = Conv2D(256, 4, strides=2)(x)
    x = LeakyReLU()(x)

    x = Conv2D(256, 4, strides=2)(x)
    x = LeakyReLU()(x)

    x = Conv2D(256, 4, strides=2)(x)
    x = LeakyReLU()(x)

    x = Conv2D(256, 4, strides=2)(x)
    x = LeakyReLU()(x)

    x = Flatten()(x)
    x = Dropout(0.4)(x)

    x = Dense(1, activation='sigmoid')(x)
    discriminator = Model(disc_input, x)

    optimizer = RMSprop(
        lr=.0001,
        clipvalue=1.0,
        decay=1e-8
    )

    discriminator.compile(
        optimizer=optimizer,
        loss='binary_crossentropy'
    )

    return discriminator

In [ ]:
from IPython.display import Image
from keras.utils.vis_utils import model_to_dot
generator = create_generator()
generator.summary()

In [ ]:
Image(model_to_dot(generator, show_shapes=True).create_png())

In [ ]:
discriminator = create_discriminator()
discriminator.trainable = False
discriminator.summary()

In [ ]:
Image(model_to_dot(discriminator, show_shapes=True).create_png())

In [ ]:
gan_input = Input(shape=(LATENT_DIM, ))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.summary()

In [ ]:
import optuna
from optuna import Trial
import numpy as np
from keras.optimizers import RMSprop

def objective(trial: Trial):
    # Các tham số không đổi
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    iters = trial.suggest_int('iters', 1000, 3000) 
    batch_size = trial.suggest_int('batch_size', 8, 16)

    # Khởi tạo optimizer
    optimizer = RMSprop(lr=lr, clipvalue=1.0, decay=1e-8)
    gan.compile(optimizer=optimizer, loss='binary_crossentropy')

    # Khởi tạo biến để lưu mất mát
    d_losses = []
    a_losses = []

    for step in range(iters):
        # Sinh dữ liệu giả và huấn luyện
        latent_vectors = np.random.normal(size=(batch_size, LATENT_DIM))
        generated = generator.predict(latent_vectors)
        real = images[start:start + batch_size]
        combined_images = np.concatenate([generated, real])

        # Huấn luyện bộ phân biệt và bộ tạo
        labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
        labels += .05 * np.random.random(labels.shape)
        d_loss = discriminator.train_on_batch(combined_images, labels)
        d_losses.append(d_loss)

        misleading_targets = np.zeros((batch_size, 1))
        a_loss = gan.train_on_batch(latent_vectors, misleading_targets)
        a_losses.append(a_loss)

    # Chọn tiêu chí đánh giá
    final_loss = np.mean(a_losses)+ np.mean(d_losses)  # hoặc np.mean(d_losses), tùy thuộc vào mục tiêu cụ thể của bạn
    return final_loss

# Tạo và chạy nghiên cứu tối ưu hóa
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  

# In kết quả tốt nhất
print('Best trial:')
trial = study.best_trial
print('Value: ', trial.value)
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

In [ ]:
import time
optimizer = RMSprop(lr=0.0002, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=optimizer, loss='binary_crossentropy')
iters = 2462
batch_size = 8
RES_DIR = 'res2'
FILE_PATH = '%s/generated_%d.png'
if not os.path.isdir(RES_DIR):
    os.mkdir(RES_DIR)

CONTROL_SIZE_SQRT = 6
control_vectors = np.random.normal(size=(CONTROL_SIZE_SQRT**2, LATENT_DIM)) / 2

start = 0
d_losses = []
a_losses = []
images_saved = 0
for step in range(iters):
    start_time = time.time()
    latent_vectors = np.random.normal(size=(batch_size, LATENT_DIM))
    generated = generator.predict(latent_vectors)

    real = images[start:start + batch_size]
    combined_images = np.concatenate([generated, real])

    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    labels += .05 * np.random.random(labels.shape)

    d_loss = discriminator.train_on_batch(combined_images, labels)
    d_losses.append(d_loss)

    latent_vectors = np.random.normal(size=(batch_size, LATENT_DIM))
    misleading_targets = np.zeros((batch_size, 1))

    a_loss = gan.train_on_batch(latent_vectors, misleading_targets)
    a_losses.append(a_loss)

    start += batch_size
    if start > images.shape[0] - batch_size:
        start = 0
    print(step)
    if step % 50 == 49:
        gan.save_weights('/gan_model.h5')

        print('%d/%d: d_loss: %.4f,  a_loss: %.4f.  (%.1f sec)' % (step + 1, iters, d_loss, a_loss, time.time() - start_time))

        control_image = np.zeros((WIDTH * CONTROL_SIZE_SQRT, HEIGHT * CONTROL_SIZE_SQRT, CHANNELS))
        control_generated = generator.predict(control_vectors)
        
        for i in range(CONTROL_SIZE_SQRT ** 2):
            x_off = i % CONTROL_SIZE_SQRT
            y_off = i // CONTROL_SIZE_SQRT
            control_image[x_off * WIDTH:(x_off + 1) * WIDTH, y_off * HEIGHT:(y_off + 1) * HEIGHT, :] = control_generated[i, :, :, :]
        im = Img.fromarray(np.uint8(control_image * 255))#.save(StringIO(), 'jpeg')
        im.save(FILE_PATH % (RES_DIR, images_saved))
        images_saved += 1
        